##Avaliação dos classificadores treinados em contexto fechado para predições em contexto aberto
###Rafael Augusto de Souza Silva - Sistemas de Informação - UNIFEI

## Importações de Bibliotecas  <a class="anchor" id="imports"></a>

In [ ]:
# importo a biblioteca pandas e a twitter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imblearn

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score
from yellowbrick.classifier import ConfusionMatrix
from yellowbrick.model_selection import FeatureImportances

##Carregamento do CSV base de contexto fechado

In [ ]:
df_treino = pd.read_csv("df_hibrido.csv",sep =";")
df_treino.head()

Verificação da quantidade de amostras por classe, humana ou bot

In [ ]:
df_treino.tipo_usuario.value_counts().plot(kind='pie') #0 é humano e 1 é bot

Seleção das colunas que possuem atributos de valor numérico

In [ ]:
colunas_treino = []
for x in df_treino.columns:
    if (df_treino[x].dtype == np.float64 or df_treino[x].dtype == np.float32 or df_treino[x].dtype == np.int64 or df_treino[x].dtype == np.int32):
        colunas_treino.append(x)

In [ ]:
colunas_treino

Seleção das colunas que não possuem dados nulos

In [ ]:
atributos_treino = [x for x in colunas_treino if df_treino[x].isnull().sum() == 0]
df_treino[colunas_treino].isnull().sum() == 0

In [ ]:
atributos_treino

Remoção de atributos que não são necessários

In [ ]:
atributos_treino.remove('contagem_tweets')
atributos_treino.remove('intervalo_tweets')
atributos_treino.remove('dp_intervalo')

Quantidade de amostras por classe

In [ ]:
df_treino.groupby(['tipo_usuario']).size()

#Separação de atributos de treinamento

Caso for realizar o treinamento utilizando os 22 atributos 

In [ ]:
X = df_treino[atributos_treino[1:23]]
y = df_treino[atributos_treino[23]]

Caso for realizar o treinamento utilizando apenas os 4 atributos mais relevantes

In [ ]:
melhores5 = ['favourites_count','n_mencoes','n_retweets','num_hashtags']
X = df_treino[melhores5]
y = df_treino[atributos_treino[23]]

#Declaração dos classificadores e separação do conjunto em treino e teste

In [ ]:
pipe_mlp = make_pipeline(StandardScaler(),MLPClassifier(random_state=42))
forest = RandomForestClassifier(random_state=42, n_jobs = -1)
rt = DecisionTreeClassifier(random_state=42)
nb = MultinomialNB()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Carregamento do CSV do conjunto de contexto aberto

In [ ]:
df_validacao = pd.read_csv("base_de_validacao.csv",sep =";")
df_validacao.head()

Verificação do balanceamento do conjunto de dados

In [ ]:
df_validacao.tipo_usuario.value_counts().plot(kind='pie') #0 é humano e 1 é bot

Seleção das colunas com atributos de valor numérico

In [ ]:
colunas_validacao = []
for x in df_validacao.columns:
    if (df_validacao[x].dtype == np.float64 or df_validacao[x].dtype == np.float32 or df_validacao[x].dtype == np.int64 or df_validacao[x].dtype == np.int32):
        colunas_validacao.append(x)

In [ ]:
colunas_validacao

Seleção das colunas que não possuem dados nulos

In [ ]:
atributos_validacao = [x for x in colunas_validacao if df_validacao[x].isnull().sum() == 0]
df_validacao[colunas_validacao].isnull().sum() == 0

Agrupamento de usuário por classe

In [ ]:
df_validacao.groupby(['tipo_usuario']).size()

Remoção do atributo desnecessário

In [ ]:
atributos_validacao.remove('contagem_tweets')

Verificação dos atributos selecionados

In [ ]:
atributos_validacao

#Classificações

##Naive bayes

In [ ]:
nb.fit(X_train, y_train)
predicoes_1 = nb.predict(X_test)
accuracy_score(y_test, predicoes_1)

matriz de confusão para o classificador

In [ ]:
cm = ConfusionMatrix(nb, classes=["Humano","bot"], cmap='GnBu')
cm.score(X_test, y_test)

Validações cruzadas de 3,5 e 10 pastas

In [ ]:
cross_val_score(nb, X_train, y_train, cv=3).mean()

In [ ]:
cross_val_score(nb, X_train, y_train, cv=5).mean()

In [ ]:
cross_val_score(nb, X_train, y_train, cv=10).mean()

Metricas de avaliação das classificações, F1, precisão e revocação respectivamente

In [ ]:
metrics.f1_score(y_test, predicoes_1)

In [ ]:
precision_score(y_test, predicoes_1)

In [ ]:
recall_score(y_test, predicoes_1)

Avaliação do naive bayes para treinamento em contexto fechado e predição em 
contexto aberto

###Se a analise levar em conta 22 atributos

In [ ]:
v_nb = nb.predict(df_validacao[atributos_validacao[1:23]])
accuracy_score(df_validacao[atributos_validacao[23]], v_nb)

In [ ]:
cm = ConfusionMatrix(nb, classes=["Humano","bot"], cmap='GnBu')
cm.score(df_validacao[atributos_validacao[1:23]], df_validacao[atributos_validacao[23]])

In [ ]:
metrics.f1_score(df_validacao[atributos_validacao[23]], v_nb)

In [ ]:
precision_score(df_validacao[atributos_validacao[23]], v_nb)

In [1]:
recall_score(df_validacao[atributos_validacao[23]], v_nb)

NameError: ignored

###Se a analise levar em conta 4 melhores atributos

In [ ]:
v_nb = nb.predict(df_validacao[melhores_atr])
accuracy_score(df_validacao[atributos_validacao[23]], v_nb)

In [ ]:
cm = ConfusionMatrix(nb, classes=["Humano","bot"], cmap='GnBu')
cm.score(df_validacao[melhores_atr], df_validacao[atributos_validacao[23]])

In [ ]:
metrics.f1_score(df_validacao[atributos_validacao[23]], v_nb)

In [ ]:
precision_score(df_validacao[atributos_validacao[23]], v_nb)

In [ ]:
recall_score(df_validacao[atributos_validacao[23]], v_nb)

##Arvore de Decisão

In [ ]:
rt.fit(X_train,y_train)
predicoes_2 = rt.predict(X_test)
accuracy_score(y_test, predicoes_2)

In [ ]:
cm = ConfusionMatrix(rt, classes=["Humano","bot"], cmap='GnBu')
cm.score(X_test, y_test)

In [ ]:
cross_val_score(rt, X_train, y_train, cv=3).mean()

In [ ]:
cross_val_score(rt, X_train, y_train, cv=5).mean()

In [ ]:
cross_val_score(rt, X_train, y_train, cv=10).mean()

In [ ]:
metrics.f1_score(y_test, predicoes_2)

In [ ]:
precision_score(y_test, predicoes_2)

In [ ]:
recall_score(y_test, predicoes_2)

###Se a analise levar em conta 22 atributos

In [ ]:
v_rt = rt.predict(df_validacao[atributos_validacao[1:23]])
accuracy_score(df_validacao[atributos_validacao[23]], v_rt)

In [ ]:
cm = ConfusionMatrix(rt, classes=["Humano","bot"], cmap='GnBu')
cm.score(df_validacao[atributos_validacao[1:23]], df_validacao[atributos_validacao[23]])

In [ ]:
metrics.f1_score(df_validacao[atributos_validacao[23]], v_rt)

In [ ]:
precision_score(df_validacao[atributos_validacao[23]], v_rt)

In [ ]:
recall_score(df_validacao[atributos_validacao[23]], v_rt)

###Se a analise levar em conta 4 melhores atributos

In [ ]:
v_rt = rt.predict(df_validacao[melhores5])
accuracy_score(df_validacao[atributos_validacao[23]], v_rt)

In [ ]:
cm = ConfusionMatrix(rt, classes=["Humano","bot"], cmap='GnBu')
cm.score(df_validacao[melhores5], df_validacao[atributos_validacao[23]])

In [ ]:
metrics.f1_score(df_validacao[atributos_validacao[23]], v_rt)

In [ ]:
precision_score(df_validacao[atributos_validacao[23]], v_rt)

In [ ]:
recall_score(df_validacao[atributos_validacao[23]], v_rt)

##Floresta Aleatória

In [ ]:
forest.fit(X_train,y_train)
predicoes_3 = forest.predict(X_test)
accuracy_score(y_test, predicoes_3)

In [ ]:
cm = ConfusionMatrix(forest, classes=["Humano","bot"], cmap='GnBu')
cm.score(X_test, y_test)

In [ ]:
cross_val_score(forest, X_train, y_train, cv=3).mean()

In [ ]:
cross_val_score(forest, X_train, y_train, cv=5).mean()

In [ ]:
cross_val_score(forest, X_train, y_train, cv=10).mean()

In [ ]:
metrics.f1_score(y_test, predicoes_3)

In [ ]:
precision_score(y_test, predicoes_3)

In [ ]:
recall_score(y_test, predicoes_3)

###Se a analise levar em conta 22 atributos

In [ ]:
v_f = forest.predict(df_validacao[atributos_validacao[1:23]])
accuracy_score(df_validacao[atributos_validacao[23]], v_f)

In [ ]:
cm = ConfusionMatrix(forest, classes=["Humano","bot"], cmap='GnBu')
cm.score(df_validacao[atributos_validacao[1:23]], df_validacao[atributos_validacao[23]])

In [ ]:
metrics.f1_score(df_validacao[atributos_validacao[23]], v_f)

In [ ]:
precision_score(df_validacao[atributos_validacao[23]], v_f)

In [ ]:
recall_score(df_validacao[atributos_validacao[23]], v_f)

###Se a analise levar em conta 4 melhores atributos

In [ ]:
v_f = forest.predict(df_validacao[melhores5])
accuracy_score(df_validacao[atributos_validacao[23]],v_f)

In [ ]:
cm = ConfusionMatrix(forest, classes=["Humano","bot"], cmap='GnBu')
cm.score(df_validacao[melhores5], df_validacao[atributos_validacao[23]])

In [ ]:
metrics.f1_score(df_validacao[atributos_validacao[23]], v_f)

In [ ]:
precision_score(df_validacao[atributos_validacao[23]], v_f)

In [ ]:
recall_score(df_validacao[atributos_validacao[23]], v_f)

##MLP

In [ ]:
pipe_mlp.fit(X_train,y_train)
predicoes_4 = pipe_mlp.predict(X_test)
accuracy_score(y_test, predicoes_4)

In [ ]:
cm = ConfusionMatrix(pipe_mlp, classes=["Humano","bot"], cmap='GnBu')
cm.score(X_test, y_test)

In [ ]:
cross_val_score(pipe_mlp, X_train, y_train, cv=3).mean()

In [ ]:
cross_val_score(pipe_mlp, X_train, y_train, cv=5).mean()

In [ ]:
cross_val_score(pipe_mlp, X_train, y_train, cv=10).mean()

In [ ]:
metrics.f1_score(y_test, predicoes_4)

In [ ]:
precision_score(y_test, predicoes_4)

In [ ]:
recall_score(y_test, predicoes_4)

###Se a analise levar em conta 22 atributos

In [ ]:
v_mlp = pipe_mlp.predict(df_validacao[atributos_validacao[1:23]])
accuracy_score(df_validacao[atributos_validacao[23]], v_mlp)

In [ ]:
cm = ConfusionMatrix(pipe_mlp, classes=["Humano","bot"], cmap='GnBu')
cm.score(df_validacao[atributos_validacao[1:23]], df_validacao[atributos_validacao[23]])

In [ ]:
metrics.f1_score(df_validacao[atributos_validacao[23]], v_mlp)

In [ ]:
precision_score(df_validacao[atributos_validacao[23]], v_mlp)

In [ ]:
recall_score(df_validacao[atributos_validacao[23]], v_mlp)

###Se a analise levar em conta 4 melhores atributos

In [ ]:
v_mlp = pipe_mlp.predict(df_validacao[melhores5])
accuracy_score(df_validacao[atributos_validacao[23]], v_mlp)

In [ ]:
cm = ConfusionMatrix(pipe_mlp, classes=["Humano","bot"], cmap='GnBu')
cm.score(df_validacao[melhores5], df_validacao[atributos_validacao[23]])

In [ ]:
metrics.f1_score(df_validacao[atributos_validacao[23]], v_mlp)

In [ ]:
precision_score(df_validacao[atributos_validacao[23]], v_mlp)

In [ ]:
recall_score(df_validacao[atributos_validacao[23]], v_mlp)